## 6.2 Conversation Buffer Memory using ECEL

<div style="text-align: right"> Initial issue : 2025.05.12 </div>
<div style="text-align: right"> last update : 2025.05.12 </div>

ECEL에 chat_history를 추가하는 방법

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.memory import ConversationBufferMemory

In [3]:
memory = ConversationBufferMemory()
memory

/tmp/ipykernel_62323/2223904900.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]))

In [4]:
memory.save_context(
    inputs={
        "human": "안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?"
    },
    outputs={
        "ai": "안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?"
    },
)
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?', additional_kwargs={}, response_metadata={})]))

memory 의 `load_memory_variables({})` 함수는 메시지 히스토리를 반환

In [5]:
print(memory.load_memory_variables({})["history"])

Human: 안녕하세요, 비대면으로 은행 계좌를 개설하고 싶습니다. 어떻게 시작해야 하나요?
AI: 안녕하세요! 계좌 개설을 원하신다니 기쁩니다. 먼저, 본인 인증을 위해 신분증을 준비해 주시겠어요?


`load_memory` 함수를 정의

In [6]:
def load_memory(_):  # 무조건 입력을 넣어야... 규칙임
    return memory.load_memory_variables({})["chat_history"]

`return_messages=True` 로 설정하면 `HumanMessage` 와 `AIMessage` 객체를 반환

In [7]:
memory = ConversationBufferMemory(return_messages=True, k=5, memory_key = "chat_history")

### Chain 생성

프롬프트 생성

In [8]:
from langchain_core.prompts import PromptTemplate

template = """
You are a helpful and concise assistant.
Use the chat history to understand the conversation flow and provide a relevant and accurate answer to the current question.

Chat History: {chat_history}
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "question"], 
    template=template + "\nQuestion: {question}"
)

llm 정의

In [9]:
from langchain_openai import ChatOpenAI

# LLM 모델을 생성합니다.
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain

In [10]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

chain = (
    {
        "question": RunnablePassthrough(),  # Pass through the question
        "chat_history": RunnableLambda(load_memory),
    }
    | prompt  # Add the prompt template
    | llm  # Pass the input to the LLM
)

### 테스트

In [11]:
content = chain.invoke("안녕하세요. 제 이름은 홍길동입니다. 잘 부탁 드립니다.").content
memory.save_context({"input": "안녕하세요. 제 이름은 홍길동입니다. 잘 부탁 드립니다."}, 
                    {"output": content})
print(content)

안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?


In [12]:
content = chain.invoke("서울 근교 여행지 3군데 추천해줘.").content
memory.save_context({"input": "서울 근교 여행지 3군데 추천해줘."}, 
                    {"output": content})
print(content)

서울 근교에서 가볼 만한 여행지 세 곳을 추천해드리겠습니다.

1. **남한산성**: 역사적인 유적지로, 아름다운 자연 경관과 함께 산책을 즐길 수 있습니다. 가벼운 등산 코스로도 좋습니다.

2. **가평**: 자연과 함께하는 다양한 액티비티를 즐길 수 있는 곳입니다. 특히, 남이섬과 아침고요수목원이 유명합니다.

3. **양평**: 두물머리와 세미원 같은 자연 명소가 있으며, 한적한 분위기에서 여유로운 시간을 보낼 수 있습니다.

이 세 곳은 서울에서 비교적 가까워 당일치기 여행으로도 적합합니다.


In [13]:
content = chain.invoke("내 이름이 뭐라고?").content
memory.save_context({"input": "내 이름이 뭐라고?"}, 
                    {"output": content})
print(content)

당신의 이름은 홍길동입니다.


In [14]:
print(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content='안녕하세요. 제 이름은 홍길동입니다. 잘 부탁 드립니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='서울 근교 여행지 3군데 추천해줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='서울 근교에서 가볼 만한 여행지 세 곳을 추천해드리겠습니다.\n\n1. **남한산성**: 역사적인 유적지로, 아름다운 자연 경관과 함께 산책을 즐길 수 있습니다. 가벼운 등산 코스로도 좋습니다.\n\n2. **가평**: 자연과 함께하는 다양한 액티비티를 즐길 수 있는 곳입니다. 특히, 남이섬과 아침고요수목원이 유명합니다.\n\n3. **양평**: 두물머리와 세미원 같은 자연 명소가 있으며, 한적한 분위기에서 여유로운 시간을 보낼 수 있습니다.\n\n이 세 곳은 서울에서 비교적 가까워 당일치기 여행으로도 적합합니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내 이름이 뭐라고?', additional_kwargs={}, response_metadata={}), AIMessage(content='당신의 이름은 홍길동입니다.', additional_kwargs={}, response_metadata={})]}
